In [1]:
from pathlib import Path
from pydantic import BaseModel
from typing import Any
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os
import duckdb
load_dotenv()

True

In [2]:
class Neo4jGraph:

    def __init__(self, neo4j_uri:str, neo4j_username:str, neo4j_password:str, db:str)->None:
        self.uri  = neo4j_uri
        self.auth = (neo4j_username, neo4j_password)
        self.db = db
        self.driver = GraphDatabase.driver(self.uri, auth=self.auth)

    def query(self, query:str, params:dict):
        with self.driver.session(database=self.db) as session:
            result = session.run(query, params)
            return [r for r in result]

class Node(BaseModel):
    id:int
    label: str | list[str]
    properties: dict[str, Any] = {}

class Relation(BaseModel):
    id:int | None
    label:str
    properties:dict[str, Any] = {}
    source: Node
    target:Node


In [3]:
def merge_rel(graph:Neo4jGraph, relations:list[Relation], source_label:str, target_label:str):
    res = graph.query(
        "UNWIND $data as row "
        f"MATCH (source_node:{source_label} {{ id: row.source.id }}) "
        f"MATCH (target_node:{target_label} {{ id: row.target.id }}) "
        "CALL apoc.merge.relationship(source_node, "
        "row.label, "
        "row.id, "
        "row.properties, "
        "target_node, "
        "row.properties ) "
        "YIELD rel "
        "RETURN rel "
        ,
        {
            "data":[
                {
                    'id': {'id': rel.id} if rel.id is not None else {},
                    'label': rel.label,
                    'source': rel.source.model_dump(),
                    'target':rel.target.model_dump(),
                    'properties': rel.properties,
                } for rel in relations
            ],
        }
    )
    return res

In [4]:
neo4j_uri = os.environ['neo4j_uri']
neo4j_username = os.environ['neo4j_username']
neo4j_password = os.environ['neo4j_password']
neo4j_dbname = os.environ['neo4j_dbname']

In [5]:
graph = Neo4jGraph(
    neo4j_uri,
    neo4j_username,
    neo4j_password,
    neo4j_dbname,
)

In [10]:
cypher = """
MATCH (n:Character)
WHERE n.site_url IS NOT NULL
WITH n
MERGE (n)-[:HAS_LINK]->(m:Link {url: n.site_url})
return id(m) AS id
"""

In [11]:
res = graph.query(cypher, {})


In [12]:
print(f"Merged {len(res)} links")

Merged 8263 links
